# Day 13 of 100

## Notes

Multiprocessing in Python:
    
    - We can run multiple tasks at the same time using library multiprocessing.
    - Doesn't work in some IDEs.

In [20]:
from random import randint
from time import time, sleep
from multiprocessing import Process
from os import getpid


def multi_download_task(filename):
    print('Start downloading process, number [%d]' % getpid())
    print('Start downloading %s...' % filename)
    time_to_download = randint(1, 5)
    sleep(time_to_download)
    print('%s finishes downloading, it takes %d seconds' % (filename, time_to_download))
    
def main():
    start = time()
    p1 = Process(target=multi_download_task, args=('Python cookbook.pdf', ))
    p1.start()
    p2 = Process(target=multi_download_task, args=('Jupyter notebook.exe', ))
    p2.start()
    p1.join()
    p2.join()
    end = time()
    print('A total of %.2f seconds.' % (end - start))


if __name__ == '__main__':
    main()
    print("This Code doesn't work in Jupyter Notebook.")

A total of 0.12 seconds.
This Code doesn't work in Jupyter Notebook.


Multiprocessing in Python:
    
    - We can inherit class Thread to our own task class.
    - Works fine in Jupyter Notebook.

In [23]:
from random import randint
from threading import Thread
from time import time, sleep


class DownloadTask(Thread):

    def __init__(self, filename):
        super().__init__()
        self._filename = filename

    def run(self):
        print('Start downloading %s...' % self._filename)
        time_to_download = randint(1, 5)
        sleep(time_to_download)
        print('%s finishes downloading, it takes %d seconds.' % (self._filename, time_to_download))


def main():
    start = time()
    t1 = DownloadTask('Python cookbook.pdf')
    t1.start()
    t2 = DownloadTask('Jupyter notebook.exe')
    t2.start()
    t1.join()
    t2.join()
    end = time()
    print('It takes %.2f seconds.' % (end - start))


if __name__ == '__main__':
    main()

Start downloading Python cookbook.pdf...
Start downloading Jupyter notebook.exe...
Python cookbook.pdf finishes downloading, it takes 2 seconds.
Jupyter notebook.exe finishes downloading, it takes 2 seconds.
It takes 2.01 seconds.


Since multiple threads share process memory, it's easy to communicate between them. However, we need to be careful because without protection, it's very easy to go wrong.

In [31]:
from time import sleep
from threading import Thread


class Account(object):

    def __init__(self):
        self._balance = 0

    def deposit(self, money):
        # Calculate new balance
        new_balance = self._balance + money
        # It takes 0.01 second to finish this process.
        sleep(0.01)
        # Modify account balance.
        self._balance = new_balance

    @property
    def balance(self):
        return self._balance


class AddMoneyThread(Thread):

    def __init__(self, account, money):
        super().__init__()
        self._account = account
        self._money = money

    def run(self):
        self._account.deposit(self._money)


def main():
    account = Account()
    threads = []
    # Create 100 threads to deposite money.
    for _ in range(100):
        t = AddMoneyThread(account, 1)
        threads.append(t)
        t.start()
    # Wait until all threads are completed and find total.
    for t in threads:
        t.join()
    print('Account Total: %d dollars.' % account.balance)


if __name__ == '__main__':
    main()

Account Total: 2 dollars


Multiple threads added 1 to 0, therefore when calculating the total, it doesn't show the account total properly. 

The bank account is called critical resource which needs to be protected properly in order for the code to work.

In [32]:
from time import sleep
from threading import Thread, Lock


class Account(object):

    def __init__(self):
        self._balance = 0
        self._lock = Lock()

    def deposit(self, money):
        # Get lock and then continue.
        self._lock.acquire()
        try:
            # Calculate new balance
            new_balance = self._balance + money
            # It takes 0.01 second to finish this process.
            sleep(0.01)
            # Modify account balance.
            self._balance = new_balance
        finally:
            # Release the key.
            self._lock.release()
            
    @property
    def balance(self):
        return self._balance


class AddMoneyThread(Thread):

    def __init__(self, account, money):
        super().__init__()
        self._account = account
        self._money = money

    def run(self):
        self._account.deposit(self._money)


def main():
    account = Account()
    threads = []
    # Create 100 threads to deposite money.
    for _ in range(100):
        t = AddMoneyThread(account, 1)
        threads.append(t)
        t.start()
    # Wait until all threads are completed and find total.
    for t in threads:
        t.join()
    print('Account Total: %d dollars.' % account.balance)


if __name__ == '__main__':
    main()

Account Total: 100 dollars
